In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import pandas as pd

In [2]:
# !Pip install selenium -U
import selenium
from selenium.webdriver.chrome.service import Service

selenium.__version__

'4.10.0'

In [3]:
USER_AGENT_LIST = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36']
DRIVER_FILE_PATH = "/Users/qunishdash/Documents/chromedriver_mac64/chromedriver"

In [4]:
def get_chrome_driver(headless_flag):
    # Set up the Selenium webdriver
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options


    service = Service(executable_path=DRIVER_FILE_PATH)

    
    chrome_options = Options()
    if headless_flag:
        # in case you want headless browser
        
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("user-agent={}".format(random.choice(USER_AGENT_LIST)))
    
    driver = webdriver.Chrome(service=service, options=chrome_options) 

    return driver

In [5]:
driver = get_chrome_driver(False)

In [6]:
collection_url = "https://www2.hm.com/en_in/home/seasonal-trending/inh-home-season-gifting-wk30.html"

In [7]:
driver.get(collection_url)

In [8]:
while True:
    try:
        print("yes")
        load_more_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.js-load-more')))
        load_more_button.click()
        WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.XPATH, '//div[@class="loading-spinner"]')))
    except Exception as e:
        print("exception ",e)
        print("No load more button")
        break

yes
yes
exception  Message: element click intercepted: Element <button class="button js-load-more " data-per-page="36" data-tracking-json-template="loadMorePLPTracking" data-tracking-type="event" data-tracking-params="LOAD_MORE_PRODUCTS|... button clicked|Product list interactions|Home : Selected : Inh - Inh_Home_Friends_Wk30 : ">Load more products</button> is not clickable at point (715, 676). Other element would receive the click: <a href="/en_in/productpage.1076718015.html" title="Cotton terry guest towel" class="item-link">...</a>
  (Session info: chrome=115.0.5790.170)
Stacktrace:
0   chromedriver                        0x0000000107ad76b8 chromedriver + 4937400
1   chromedriver                        0x0000000107aceb73 chromedriver + 4901747
2   chromedriver                        0x000000010768c616 chromedriver + 435734
3   chromedriver                        0x00000001076d7b97 chromedriver + 744343
4   chromedriver                        0x00000001076d5450 chromedriver + 734288


In [10]:
from selenium.webdriver.common.by import By

all_cards = driver.find_elements(By.CSS_SELECTOR, '.hm-product-item')

In [11]:
# list(all_cards)

In [12]:
def get_card_data(card):
    try:
        title = driver.find_element(By.CLASS_NAME, "item-heading").text
    except Exception as e:
        title = ''
    
    try:
        price = driver.find_element(By.CLASS_NAME, "regular").text
    except Exception as e:
        price = ''
    
    try:
        image_url = driver.find_element(By.CLASS_NAME, "item-image").get_attribute("src")
    except Exception as e:
        image_url = ''
    
    try:
        product_url = driver.find_element(By.CLASS_NAME, "link").get_attribute("href")
    except Exception as e:
        product_url = ''
    
    try:
        available_colors = driver.find_element(By.CLASS_NAME, "list-swatches").text.replace("\n", ", ")
    except Exception as e:
        available_colors = '' 
        
    result = {
        "title": title,
        "price": price,
        "image_url": image_url,
        "product_url": product_url,
        "available_colors": available_colors
    }
    
    return result

In [13]:
all_data = []

In [14]:
for card in all_cards:
    card_result = get_card_data(card)
    all_data.append(card_result)

In [15]:
all_data[:2]

[{'title': '2-pack round table mats',
  'price': 'Rs.399.00',
  'image_url': 'https://lp2.hm.com/hmgoepprod?set=source[/f2/6e/f26ea7c4e18ba2266d380b7c48369c14b4bfd68c.jpg],origin[dam],category[],type[LOOKBOOK],res[m],hmver[1]&call=url[file:/product/style]',
  'product_url': 'https://www2.hm.com/en_in/productpage.1018932006.html',
  'available_colors': 'Greige, Anthracite grey, Light beige, Beige'},
 {'title': '2-pack round table mats',
  'price': 'Rs.399.00',
  'image_url': 'https://lp2.hm.com/hmgoepprod?set=source[/f2/6e/f26ea7c4e18ba2266d380b7c48369c14b4bfd68c.jpg],origin[dam],category[],type[LOOKBOOK],res[m],hmver[1]&call=url[file:/product/style]',
  'product_url': 'https://www2.hm.com/en_in/productpage.1018932006.html',
  'available_colors': 'Greige, Anthracite grey, Light beige, Beige'}]

In [16]:
len(all_data)

72

In [17]:
import pymongo

MONGODB_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "h&m_data_db"
COLLECTION_NAME = "home_essentials"

def push_data_to_mongodb(data):
    try:
        client = pymongo.MongoClient(MONGODB_URI)
        db = client[DATABASE_NAME]
        collection = db[COLLECTION_NAME]
        collection.insert_many(data)
        print("Data pushed to MongoDB successfully.")
    except Exception as e:
        print("Error while pushing data to MongoDB:", e)
    finally:
        client.close()

push_data_to_mongodb(all_data)

Data pushed to MongoDB successfully.


In [15]:
# create dataframe from result
df = pd.DataFrame(all_data)
print(df.shape)
df.head()

(72, 5)


,title,price,image_url,product_url,avaialable_colors
0,,,,,
1,,,,,
2,,,,,
3,,,,,
4,,,,,


In [ ]:
# save file to local
df.to_excel("h&m.xlsx",header=True,index=False)

In [15]:
driver.close()